In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas
from sklearn.linear_model import LinearRegression

In [20]:
df = pd.read_csv('Initial_Breakdown_Plan.xlsx - Dataset.csv')

In [31]:
df_location = df['Location']
unique_locations = df_location.unique()


array(['London', 'Belfast', 'Reading', 'Birmingham', 'Leeds', 'Bristol',
       'Manchester', 'Watford', 'Edinburgh', 'Glasgow', 'Cardiff'],
      dtype=object)

In [34]:
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

# List of cities in the UK
cities = ['London', 'Manchester', 'Birmingham', 'Glasgow', 'Liverpool']  # Add more cities as needed

# Initialize geocoder with scheme='http'
geolocator = Nominatim(user_agent="my_geocoder", scheme='http')

# Geocode each city to get its coordinates
city_coordinates = [(city, geolocator.geocode(city)) for city in cities]

# Create a GeoDataFrame from the city coordinates
city_gdf = gpd.GeoDataFrame(geometry=[point for city, point in city_coordinates], index=cities, columns=['geometry'])

# Load the UK map data
uk_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Plot the UK map
fig, ax = plt.subplots(figsize=(10, 8))
uk_map.plot(ax=ax, color='lightgrey', edgecolor='black')

# Plot the cities on the map
city_gdf.plot(ax=ax, color='red', markersize=50, marker='o')

# Add title
plt.title('Map of the UK with Highlighted Cities')

# Show the plot
plt.show()


GeocoderServiceError: [SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] unsafe legacy renegotiation disabled (_ssl.c:1000)